### Start

In [1]:
import numpy as np
from osgeo import gdal
from osgeo import osr
import time
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from tensorflow.keras import backend as K

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(devices=physical_devices[0], device_type='GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
keras.backend.set_image_data_format('channels_last')

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
#自定义matries 计算r2
def R2(y_true, y_pred):
    sst = K.sum(K.square(y_true - K.mean(y_true)))
    ssr = K.sum(K.square(y_pred - y_true))
    R2 = 1 - ssr/sst
    return R2

## Helper functions

In [6]:
def Array2Image(data,lats,lons):
  # get the unique coordinates
  uniqueLats = np.unique(lats)
  uniqueLons = np.unique(lons)
 
  # get number of columns and rows from coordinates
  ncols = len(uniqueLons)
  nrows = len(uniqueLats)
 
  # determine pixelsizes
  ys = uniqueLats[1] - uniqueLats[0]  
  xs = uniqueLons[1] - uniqueLons[0]
 
  # create an array with dimensions of image
  # 这里把缺失值设置成了0，之后需要修改
  arr = np.zeros([nrows, ncols], np.float32) #-9999
  arr.fill(np.nan)
 
  # fill the array with values
  for counter in range(0,len(data),1):
    index_lon = np.where(uniqueLons == lons[counter])[0][0]
    index_lat = np.where(uniqueLats == lats[counter])[0][0]
    arr[len(uniqueLats)-1-index_lat,index_lon] = data[counter]
  return arr

In [7]:
#需要得到 4*19
def ReadData(df:pd.DataFrame):
    allData = []
    feature_name = []
    for j in range(df.shape[1]):
        for i in range(df.shape[0]):
            #print(df.iloc[i,j])
            data_i = gdal.Open(df.iloc[i,j])
            band_i = data_i.GetRasterBand(1)
            array_i = band_i.ReadAsArray()
            #array_i[array_i == noData] = np.nan
            allData.append(np.reshape(array_i,(-1)))
            feature_name.append(df.columns.tolist()[j]+'_'+str(i+1))
        # (F,T,H,W)
        #allData.append(data)
    allData = np.array(allData,dtype = np.float64)
    #print(feature_name)
    return  pd.DataFrame(allData.T,columns=feature_name)

In [8]:
templete  = gdal.Open('F:/SGYL/SM_Downscaling2/Lon.tif')
def convert2Tif(data,outputPath):
    driver = gdal.GetDriverByName('GTiff')
    tif = driver.Create(outputPath,data.shape[1],data.shape[0],1,gdal.GDT_Int16)
    #geotransform = (73.49117339381806,0.008983152841195215,0.0,39.831299697859585,0.0,-0.008983152841195215)
    tif.SetGeoTransform(templete.GetGeoTransform())
    tif.SetProjection(templete.GetProjection())
    tif.GetRasterBand(1).WriteArray(data)
    #tif.GetRasterBand(1).SetNoDataValue()
    tif.FlushCache()
    del tif


## Preprocessing

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
noData_i = gdal.Open('F:/SGYL/SM_results_data/DATA/data_2020/LST_Diff/LST_Diff_2020_1.tif')
noData_band = noData_i.GetRasterBand(1)
noData_arr = noData_band.ReadAsArray()
noData = np.min(noData_arr)
print(noData)

-3.4e+38


In [17]:
feature_order = ['NDVI','EVI','LST','LST_Diff','Pre','SWCI','VSDI','SIWSI','ET'] #
static_feature = ['TWI','dem','slope','clay','sand','silt','Lon','Lat']

year = 2001

root = 'F:/SGYL/SM_results_data/DATA/data_'+str(year)+'/'

predictor_path = dict()



for dir in feature_order:
    path = os.path.join(root,dir)
    if dir == 'ET':
        before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',i) for i in os.listdir('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/')][-3:]
        files = before + [os.path.join(path+'/',i) for i in os.listdir(path+'/')]
    elif year in [2004,2008,2012,2016,2020]:
        before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,366)][-3:]
        files = before + [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,367)]
    elif year in [2005,2009,2013,2017,2021]:
        before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,367)][-3:]
        files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]
    elif year == 2001:
        before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,4)][-3:]
        files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,362)]
    else :
        before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,366)][-3:]
        files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]        
    predictor_path[dir] = files
predictor_path['ET']

path_length = len(predictor_path['LST'])

for var in static_feature:
    path_i = os.path.join('F:/SGYL/SM_results_data/Static/',var+'.tif')
    files = [path_i for i in range(path_length)]
    predictor_path[var] = files

predictor_path = pd.DataFrame(predictor_path)

print(predictor_path.iloc[1,1])


F:/SGYL/SM_results_data/DATA/data_2000/EVI/EVI_2000_2.tif


In [12]:
test = ReadData(predictor_path.iloc[0:4,])
print(test.shape)
print(test.columns.tolist())

(5350352, 68)
['NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4', 'EVI_1', 'EVI_2', 'EVI_3', 'EVI_4', 'LST_1', 'LST_2', 'LST_3', 'LST_4', 'LST_Diff_1', 'LST_Diff_2', 'LST_Diff_3', 'LST_Diff_4', 'Pre_1', 'Pre_2', 'Pre_3', 'Pre_4', 'SWCI_1', 'SWCI_2', 'SWCI_3', 'SWCI_4', 'VSDI_1', 'VSDI_2', 'VSDI_3', 'VSDI_4', 'SIWSI_1', 'SIWSI_2', 'SIWSI_3', 'SIWSI_4', 'ET_1', 'ET_2', 'ET_3', 'ET_4', 'TWI_1', 'TWI_2', 'TWI_3', 'TWI_4', 'dem_1', 'dem_2', 'dem_3', 'dem_4', 'slope_1', 'slope_2', 'slope_3', 'slope_4', 'clay_1', 'clay_2', 'clay_3', 'clay_4', 'sand_1', 'sand_2', 'sand_3', 'sand_4', 'silt_1', 'silt_2', 'silt_3', 'silt_4', 'Lon_1', 'Lon_2', 'Lon_3', 'Lon_4', 'Lat_1', 'Lat_2', 'Lat_3', 'Lat_4']


In [26]:
'NDVI'.title()

'Ndvi'

## Predict

In [18]:
#['NDVI', 'EVI', 'LST', 'LST_Diff', 'Pre', 'SWCI', 'VSDI', 'SIWSI', 'ET', 'TWI', 'dem', 'aspect', 'slope', 'clay', 'sand', 'silt', 'Lon', 'Lat']
error_index_year = []
error_index_day = []
batch_size = 4096*4
for year in [2001]:
    try:
        print(year,'************************************************')
        print('load model')
        model_path = os.path.join('D:/SGYL/SM_results_data/check_points/LSTM/','LSTM_'+str(year)+'_best.hdf5')
        print(model_path)
        model = keras.models.load_model(model_path,custom_objects={'R2':R2})

        print('load training data')
        #load training data
        data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split_LSTM/','train_LSTM_data_'+str(year)+'.csv'))
        X_train = data_train.drop(['SM','Aspect_1','Aspect_2','Aspect_3','Aspect_4'],axis = 1)
        X_train_columns = X_train.columns.to_list()
        print(X_train.columns.to_list())
        standarder = StandardScaler()
        X_train = standarder.fit_transform(X_train)

        print('prepare file path')
        # prepare data path
        feature_order = ['NDVI','EVI','LST','LST_Diff','Pre','SWCI','VSDI','SIWSI','ET'] #
        static_feature = ['TWI','dem','slope','clay','sand','silt','Lon','Lat']

        root = 'F:/SGYL/SM_results_data/DATA/data_'+str(year)+'/'

        predictor_path = dict()



        for dir in feature_order:
            path = os.path.join(root,dir)
            if dir == 'ET':
                before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',i) for i in os.listdir('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/')][-3:]
                files = before + [os.path.join(path+'/',i) for i in os.listdir(path+'/')]
            elif year in [2004,2008,2012,2016,2020]:
                before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,366)][-3:]
                files = before + [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,367)]
            elif year in [2005,2009,2013,2017,2021]:
                before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,367)][-3:]
                files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]
            elif year == 2002 :
                before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,362)][-3:]
                files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]
            elif year == 2001:
                before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,4)][-3:]
                files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,362)]
            else :
                before = [os.path.join('F:/SGYL/SM_results_data/DATA/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,366)][-3:]
                files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]            
            predictor_path[dir] = files
        predictor_path['ET']

        path_length = len(predictor_path['LST'])

        for var in static_feature:
            path_i = os.path.join('F:/SGYL/SM_results_data/Static/',var+'.tif')
            files = [path_i for i in range(path_length)]
            predictor_path[var] = files

        predictor_path = pd.DataFrame(predictor_path)
        predictor_path.columns = feature_order+static_feature

        print('start predict','************************************************')       
        
        for i in range(3,predictor_path.shape[0]):#predictor_path.shape[0] LSTM从3开始
            try:
                #print(i,'************************************************')
                data_i = ReadData(predictor_path.iloc[(i-3):(i+1),:]) 
                lon_lat = data_i[['Lon_1','Lat_1']]
                lon_lat.columns = ['Lon','Lat']
                data_i['DOY_1'] = int(i+1-6)
                data_i['DOY_2'] = int(i+1-5)
                data_i['DOY_3'] = int(i+1-4)
                data_i['DOY_4'] = int(i+1-3)
                data_i.columns = X_train_columns
                data_i = data_i.to_numpy()
                data_i[data_i == noData] = np.nan
                    #print(data_i.shape)
                    #dealing with missing values median
                for index in range(data_i.shape[1]):
                    median_i = np.nanmedian(data_i[:,index])
                    data_i[:,index][np.isnan(data_i[:,index])] = median_i
                data_i = pd.DataFrame(data_i,columns=X_train_columns)
                data_i = standarder.transform(data_i)
                # reshape to (,4,19)
                data_i = data_i.reshape((-1,18,4))
                data_i = np.moveaxis(data_i,2,1)
                data_ii = [data_i[i*batch_size:(i+1)*batch_size,:,:] for i in range(data_i.shape[0]//batch_size+1)]
                data_pred = np.concatenate([model(i,training = False).numpy() for i in data_ii])#model.predict(data_i,batch_size=4096)
                #print(data_pred.shape)
                predicted_arr = Array2Image(data_pred,lon_lat['Lat'],lon_lat['Lon'])
                save_path = 'F:/SGYL/SM_results_data/results/'+str(year)+'/'+'LSTM'+'/'+'LSTM_'+str(year)+'_'+str(i+1-3)+'.tif'
                predicted_arr = predicted_arr * 10000
                convert2Tif(predicted_arr,save_path)
                print(i-3,': save the %d tif successfully for the year %d '%(i+1-3,year))
            except Exception as e:
                print('failed at year index %d'%(year,i+1-3)) # 这里的index是第几天
                print(repr(e))
                error_index_day.append((year,i))
            continue
    except:
        print('failed at index %d'%(i+1-3)) # 这里的index是第几天
        error_index_year.append((year,i))
    continue
    

2001 ************************************************
load model
D:/SGYL/SM_results_data/check_points/LSTM/LSTM_2001_best.hdf5
load training data
['NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4', 'EVI_1', 'EVI_2', 'EVI_3', 'EVI_4', 'LST_1', 'LST_2', 'LST_3', 'LST_4', 'LST_Diff_1', 'LST_Diff_2', 'LST_Diff_3', 'LST_Diff_4', 'Pre_1', 'Pre_2', 'Pre_3', 'Pre_4', 'SWCI_1', 'SWCI_2', 'SWCI_3', 'SWCI_4', 'VSDI_1', 'VSDI_2', 'VSDI_3', 'VSDI_4', 'SIWSI_1', 'SIWSI_2', 'SIWSI_3', 'SIWSI_4', 'ET_1', 'ET_2', 'ET_3', 'ET_4', 'TWI_1', 'TWI_2', 'TWI_3', 'TWI_4', 'Dem_1', 'Dem_2', 'Dem_3', 'Dem_4', 'Slope_1', 'Slope_2', 'Slope_3', 'Slope_4', 'Clay_1', 'Clay_2', 'Clay_3', 'Clay_4', 'Sand_1', 'Sand_2', 'Sand_3', 'Sand_4', 'Silt_1', 'Silt_2', 'Silt_3', 'Silt_4', 'Lon_1', 'Lon_2', 'Lon_3', 'Lon_4', 'Lat_1', 'Lat_2', 'Lat_3', 'Lat_4', 'DOY_1', 'DOY_2', 'DOY_3', 'DOY_4']
prepare file path
start predict ************************************************
0 : save the 1 tif successfully for the year 2001 
1 : save the 2 ti

In [19]:
print(error_index_year)
print(error_index_day)

[]
[]


In [16]:
predictor_path.iloc[3,1]

'F:/SGYL/SM_results_data/DATA/data_2017/EVI/EVI_2017_1.tif'

In [ ]:
year = 2017
print(year,'************************************************')
print('load model')
model_path = os.path.join('D:/SGYL/SM_results_data/check_points/LSTM/','LSTM_'+str(year)+'_best.hdf5')
print(model_path)
model = keras.models.load_model(model_path,custom_objects={'R2':R2})

print('load training data')
        #load training data
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_results_data/Train/split_LSTM/','train_LSTM_data_'+str(year)+'.csv'))
X_train = data_train.drop(['SM','Aspect_1','Aspect_2','Aspect_3','Aspect_4'],axis = 1)
X_train_columns = X_train.columns.to_list()
print(X_train.columns.to_list())
standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)

print('prepare file path')
        # prepare data path
feature_order = ['NDVI','EVI','LST','LST_Diff','Pre','SWCI','VSDI','SIWSI','ET'] #
static_feature = ['TWI','dem','slope','clay','sand','silt','Lon','Lat']

root = 'F:/data_'+str(year)+'/'

predictor_path = dict()



for dir in feature_order:
    path = os.path.join(root,dir)
    if dir == 'ET':
        before = [os.path.join('F:/data_'+str(year-1)+'/'+dir+'/',i) for i in os.listdir('F:/data_'+str(year-1)+'/'+dir+'/')][-3:]
        files = before + [os.path.join(path+'/',i) for i in os.listdir(path+'/')]
    elif year in [2004,2008,2012,2016,2020]:
        before = [os.path.join('F:/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,366)][-3:]
        files = before + [os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,367)]
    elif year in [2005,2009,2013,2017,2021]:
        before = [os.path.join('F:/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,367)][-3:]
        files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]
    else :
        before = [os.path.join('F:/data_'+str(year-1)+'/'+dir+'/',dir+'_'+str(year-1)+'_'+str(i)+'.tif') for i in range(1,366)][-3:]
        files = before+[os.path.join(path+'/',dir+'_'+str(year)+'_'+str(i)+'.tif') for i in range(1,366)]        
    predictor_path[dir] = files
predictor_path['ET']

path_length = len(predictor_path['LST'])

for var in static_feature:
    path_i = os.path.join('D:/SGYL/SM_results_data/Static/',var+'.tif')
    files = [path_i for i in range(path_length)]
    predictor_path[var] = files

predictor_path = pd.DataFrame(predictor_path)
predictor_path.columns = feature_order+static_feature

print('start predict','************************************************')       
        
for i in range(4,predictor_path.shape[0]):#predictor_path.shape[0] LSTM从4开始
                #print(i,'************************************************')
    data_i = ReadData(predictor_path.iloc[(i-4):i,:]) 
    lon_lat = data_i[['Lon_1','Lat_1']]
    lon_lat.columns = ['Lon','Lat']
    data_i['DOY_1'] = int(i+1-7)
    data_i['DOY_2'] = int(i+1-6)
    data_i['DOY_3'] = int(i+1-5)
    data_i['DOY_4'] = int(i+1-4)
    data_i.columns = X_train_columns
    data_i = data_i.to_numpy()
    data_i[data_i == noData] = np.nan
                    #print(data_i.shape)
                    #dealing with missing values median
    for index in range(data_i.shape[1]):
        median_i = np.nanmedian(data_i[:,index])
        data_i[:,index][np.isnan(data_i[:,index])] = median_i
    data_i = pd.DataFrame(data_i,columns=X_train_columns)
    data_i = standarder.transform(data_i)
                # reshape to (,4,19)
    data_i = data_i.reshape((-1,19,4))
    data_i = np.moveaxis(data_i,2,1)
    data_pred = model.predict(data_i,batch_size=4096)
                #print(data_pred.shape)
    predicted_arr = Array2Image(data_pred,lon_lat['Lat'],lon_lat['Lon'])
    save_path = 'D:/SGYL/SM_results_data/results/'+str(year)+'/'+'LSTM'+'/'+str(year)+'_'+str(i+1-4)+'.tif'
    convert2Tif(predicted_arr,save_path)
    print(i,': save the %d tif successfully for the year %d '%(i+1-4,year))

### here

In [ ]:
test = gdal.Open('E:/SM_results_2/25km_ESACCI/allyear/clipped/LST/LST_D/LST_D_final_2001_2001.tif')

In [ ]:
alldata = []
for i in range(test.RasterCount):
    band_i = test.GetRasterBand((i+1))
    array_i = band_i.ReadAsArray()
    alldata.append(array_i)
alldata = np.array(alldata,dtype= np.float64)
alldata.shape

In [ ]:
np.where(np.isnan(alldata))